In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\ameli\anaconda3\envs\Midbootcamp\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Json to INE

### Demand side

#### Estadística: Número de turistas según comunidad autónoma de destino principal
Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=10822

Primer paso: leer el archivo json en un dataframe inicial. Como los datos están nested en diccionarios, vamos a sacar los datos para que sean leíbles. Además, solo nos interesan los datos base (el resto de operaciones las haremos más adelante). 

In [2]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/10823?date=20151001:20220601'
turism_ccaa_destiny = pd.read_json(url)
turism_ccaa_destiny.head(50)

# Salvamos el dataset sucio como Excel
file_name = 'turists_ccaa_destiny.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_ccaa_destiny.to_excel(path+file_name)

Solo nos interesa obtener los datos base de cada ccaa, así que vamos a quedarnos con esas filas.

In [3]:
turism_ccaa_destiny = turism_ccaa_destiny.loc[turism_ccaa_destiny['Nombre'].str.contains('base')].reset_index(drop=True)
turism_ccaa_destiny.head()

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,FREG286,Turista. Total Nacional. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,FREG351,Andalucía. Turista. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,FREG350,"Balears, Illes. Turista. Dato base.",3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,FREG349,Canarias. Turista. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,FREG348,Cataluña. Turista. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


Ahora vamos a generar un nuevo diccionario que aglutine las filas que nos interesan y el que saquemos los datos de cada diccionario.

In [4]:
# %%writefile functions.py

import datetime
import pandas as pd

def normalize_json(initial_df):
    
    # Function to change date format
    def change_date_format(row):
        date = datetime.date.fromtimestamp(row//1000)
        return date

    # Create new df
    df = pd.DataFrame()
    
    # Normalize json nested dictionary from original dataset
    for col in initial_df.index:
        data = pd.json_normalize(initial_df.Data[col])
        data['Origin'] = initial_df.Nombre[col]
        df = pd.concat([df, data])
     
    # Change column order
    col = df.pop('Origin')
    df.insert(loc= 0 , column= 'Origin', value= col)
    
    # Drop innecesary columns
    df = df.drop(['Anyo', 'FK_TipoDato', 'FK_Periodo', 'Secreto'], axis=1)
    
    # Change dateformat
    df['Fecha'] = df['Fecha'].apply(change_date_format)
    
    # Reset index
    df = df.reset_index(drop=True)

    return df



In [5]:
from functions import normalize_json

turists_ccaa_origin_filtered = normalize_json(turism_ccaa_destiny)

In [6]:
turists_ccaa_origin_filtered.head()

,Origin,Fecha,Valor
0,Turista. Total Nacional. Dato base.,2022-06-01,7460277.0
1,Turista. Total Nacional. Dato base.,2022-05-01,6966913.0
2,Turista. Total Nacional. Dato base.,2022-04-01,6102142.0
3,Turista. Total Nacional. Dato base.,2022-03-01,4032127.0
4,Turista. Total Nacional. Dato base.,2022-02-01,3156809.0


Ahora vamos a:

- Quitar las columnas que no nos interesan
- Cambiar la columna origen al inicio del dataset
- Reestablecer el índice
- Cambiar el formato de la fecha
- Buscar NAN y duplicados
- Guardar el dataset

In [7]:
turists_ccaa_origin_filtered['Origin'].value_counts()

turists_ccaa_origin_filtered['Origin'] = turists_ccaa_origin_filtered['Origin'].str.replace('Turista. ', '', regex = False)
turists_ccaa_origin_filtered['Origin'] = turists_ccaa_origin_filtered['Origin'].str.replace('Dato base. ', '', regex = False)
turists_ccaa_origin_filtered['Origin'] = turists_ccaa_origin_filtered['Origin'].str.replace('.', '', regex = False)

In [8]:
turists_ccaa_origin_filtered.head()

,Origin,Fecha,Valor
0,Total Nacional,2022-06-01,7460277.0
1,Total Nacional,2022-05-01,6966913.0
2,Total Nacional,2022-04-01,6102142.0
3,Total Nacional,2022-03-01,4032127.0
4,Total Nacional,2022-02-01,3156809.0


In [9]:
turists_ccaa_origin_filtered.isna().sum()

Origin    0
Fecha     0
Valor     0
dtype: int64

In [10]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turists_ccaa_destiny.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turists_ccaa_origin_filtered.to_excel(path+file_name)

### Estadística: Número de turistas según país de residencia
Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=10822

In [11]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/10822?date=20151001:20220601'
turism_origin = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turists_origin.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_origin.to_excel(path+file_name)

In [12]:
turism_origin.head(50)

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,FREG337,Total. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,FREG321,Total. Turista. Tasa de variación anual. Total...,135,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,FREG1173,Total. Turista. Total Nacional. Acumulado en l...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,FREG1189,Total. Turista. Total Nacional. Tasa de variac...,135,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,FREG336,Alemania. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
5,FREG320,Alemania. Turista. Tasa de variación anual. To...,135,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
6,FREG1172,Alemania. Turista. Total Nacional. Acumulado e...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
7,FREG1188,Alemania. Turista. Total Nacional. Tasa de var...,135,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
8,FREG335,Bélgica. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
9,FREG319,Bélgica. Turista. Tasa de variación anual. Tot...,135,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


In [13]:
turism_origin = turism_origin.loc[turism_origin['Nombre'].str.contains('base')].reset_index(drop=True)
turism_origin.head()

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,FREG337,Total. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,FREG336,Alemania. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,FREG335,Bélgica. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,FREG334,Francia. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,FREG333,Irlanda. Turista. Dato base. Total Nacional.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


In [14]:
turism_origin_filtered = normalize_json(turism_origin)
turism_origin_filtered.head()

,Origin,Fecha,Valor
0,Total. Turista. Dato base. Total Nacional.,2022-06-01,7460277.0
1,Total. Turista. Dato base. Total Nacional.,2022-05-01,6966913.0
2,Total. Turista. Dato base. Total Nacional.,2022-04-01,6102142.0
3,Total. Turista. Dato base. Total Nacional.,2022-03-01,4032127.0
4,Total. Turista. Dato base. Total Nacional.,2022-02-01,3156809.0


In [15]:
turism_origin_filtered['Origin'].value_counts()

Total. Turista. Dato base. Total Nacional.                         81
Alemania. Turista. Dato base. Total Nacional.                      81
Bélgica. Turista. Dato base. Total Nacional.                       81
Francia. Turista. Dato base. Total Nacional.                       81
Irlanda. Turista. Dato base. Total Nacional.                       81
Italia. Turista. Dato base. Total Nacional.                        81
Países Bajos. Turista. Dato base. Total Nacional.                  81
Países Nórdicos. Turista. Dato base. Total Nacional.               81
Portugal. Turista. Dato base. Total Nacional.                      81
Reino Unido. Turista. Dato base. Total Nacional.                   81
Suiza. Turista. Dato base. Total Nacional.                         81
Europa menos UE28. Turista. Dato base. Total Nacional.             81
Estados Unidos de América. Turista. Dato base. Total Nacional.     81
Resto América. Turista. Dato base. Total Nacional.                 81
Resto del Mundo. Tur

In [16]:
turism_origin_filtered['Origin'] = turism_origin_filtered['Origin'].apply(lambda string: string.split(".")[0])

turism_origin_filtered.head()

,Origin,Fecha,Valor
0,Total,2022-06-01,7460277.0
1,Total,2022-05-01,6966913.0
2,Total,2022-04-01,6102142.0
3,Total,2022-03-01,4032127.0
4,Total,2022-02-01,3156809.0


In [17]:
turism_origin_filtered.isna().sum()

Origin     0
Fecha      0
Valor     10
dtype: int64

## HOW TO HANDLE THESE ROWS?? DROP THESE VALUES

In [18]:
rows_to_drop = turism_origin_filtered.loc[turism_origin_filtered['Valor'].isna()].index
turism_origin_filtered = turism_origin_filtered.drop(rows_to_drop, axis=0)

turism_origin_filtered.isna().sum()

Origin    0
Fecha     0
Valor     0
dtype: int64

In [19]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turists_origin.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_origin_filtered.to_excel(path+file_name)

### Estadística: Número de turistas según tipo de alojamiento principal

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=10826

In [20]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/10826?date=20151001:20220601'
        
turism_accomodation = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turists_accomodation.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_accomodation.to_excel(path+file_name)

In [21]:
turism_accomodation.Nombre.value_counts()

Turista. Total. Total Nacional. Dato base.                                                   1
Turista. Total. Total Nacional. Tasa de variación anual.                                     1
Turista. No de mercado. Total Nacional. Dato base.                                           1
Turista. No de mercado. Total Nacional. Tasa de variación anual.                             1
Turista. No de mercado. Total Nacional. Acumulado en lo que va de año.                       1
Turista. No de mercado. Total Nacional. Tasa de variación acumulada.                         1
Turista. Vivienda en propiedad. Total Nacional. Dato base.                                   1
Turista. Vivienda en propiedad. Total Nacional. Tasa de variación anual.                     1
Turista. Vivienda en propiedad. Total Nacional. Acumulado en lo que va de año.               1
Turista. Vivienda en propiedad. Total Nacional. Tasa de variación acumulada.                 1
Turista. Vivienda de familiares o amigos. Total Na

In [22]:
turism_accomodation = turism_accomodation.loc[turism_accomodation['Nombre'].str.contains('base')].reset_index(drop=True)
turism_accomodation.head()

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,FREG305,Turista. Total. Total Nacional. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,FREG593,Turista. De mercado. Total Nacional. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,FREG304,Turista. Hoteles o apartahotel. Total Nacional...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,FREG665,Turista. Vivienda en alquiler. Total Nacional....,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,FREG303,Turista. Resto de mercado. Total Nacional. Dat...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


In [23]:
turism_accomodation_filtered = normalize_json(turism_accomodation)
turism_accomodation_filtered.head()

,Origin,Fecha,Valor
0,Turista. Total. Total Nacional. Dato base.,2022-06-01,7460277.0
1,Turista. Total. Total Nacional. Dato base.,2022-05-01,6966913.0
2,Turista. Total. Total Nacional. Dato base.,2022-04-01,6102142.0
3,Turista. Total. Total Nacional. Dato base.,2022-03-01,4032127.0
4,Turista. Total. Total Nacional. Dato base.,2022-02-01,3156809.0


In [24]:
turism_accomodation_filtered['Origin'].value_counts()

Turista. Total. Total Nacional. Dato base.                               81
Turista. De mercado. Total Nacional. Dato base.                          81
Turista. Hoteles o apartahotel. Total Nacional. Dato base.               81
Turista. Vivienda en alquiler. Total Nacional. Dato base.                81
Turista. Resto de mercado. Total Nacional. Dato base.                    81
Turista. No de mercado. Total Nacional. Dato base.                       81
Turista. Vivienda en propiedad. Total Nacional. Dato base.               81
Turista. Vivienda de familiares o amigos. Total Nacional. Dato base.     81
Turista. Resto de no mercado. Total Nacional. Dato base.                 81
Name: Origin, dtype: int64

In [25]:
turism_accomodation_filtered['Origin'] = turism_accomodation_filtered['Origin'].apply(lambda string: string.split(".")[1])
turism_accomodation_filtered['Origin'].value_counts()

 Total                              81
 De mercado                         81
 Hoteles o apartahotel              81
 Vivienda en alquiler               81
 Resto de mercado                   81
 No de mercado                      81
 Vivienda en propiedad              81
 Vivienda de familiares o amigos    81
 Resto de no mercado                81
Name: Origin, dtype: int64

In [26]:
turism_accomodation_filtered.isna().sum()

Origin    0
Fecha     0
Valor     0
dtype: int64

In [27]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turists_accomodation.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_accomodation_filtered.to_excel(path+file_name)

### Estadística: Número de visitantes según duración de la estancia

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=14290

In [28]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/14290?date=20151001:20220601'

turism_stay = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turists_stay.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_stay.to_excel(path+file_name)

In [29]:
turism_stay.Nombre.unique()

array(['Total. Total Nacional. Ninguna noche. Dato base. ',
       'Total. Total Nacional. Ninguna noche. Tasa de variación anual. ',
       'Total. Total Nacional. Ninguna noche. Acumulado en lo que va de año. ',
       'Total. Total Nacional. Ninguna noche. Tasa de variación acumulada. ',
       'Total. Total Nacional. 1 noche. Dato base. ',
       'Total. Total Nacional. 1 noche. Tasa de variación anual. ',
       'Total. Total Nacional. 1 noche. Acumulado en lo que va de año. ',
       'Total. Total Nacional. 1 noche. Tasa de variación acumulada. ',
       'Total. Total Nacional. De 2 a 3 noches. Dato base. ',
       'Total. Total Nacional. De 2 a 3 noches. Tasa de variación anual. ',
       'Total. Total Nacional. De 2 a 3 noches. Acumulado en lo que va de año. ',
       'Total. Total Nacional. De 2 a 3 noches. Tasa de variación acumulada. ',
       'Total. Total Nacional. De 4 a 7 noches. Dato base. ',
       'Total. Total Nacional. De 4 a 7 noches. Tasa de variación anual. ',
  

In [30]:
turism_stay = turism_stay.loc[turism_stay['Nombre'].str.contains('base')].reset_index(drop=True)
turism_stay.head()

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,FREG847,Total. Total Nacional. Ninguna noche. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,FREG846,Total. Total Nacional. 1 noche. Dato base.,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,FREG845,Total. Total Nacional. De 2 a 3 noches. Dato b...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,FREG844,Total. Total Nacional. De 4 a 7 noches. Dato b...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,FREG843,Total. Total Nacional. De 8 a 15 noches. Dato ...,3,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


In [31]:
turism_stay_filtered = normalize_json(turism_stay)
turism_stay_filtered.head()

,Origin,Fecha,Valor
0,Total. Total Nacional. Ninguna noche. Dato base.,2022-06-01,2635978.0
1,Total. Total Nacional. Ninguna noche. Dato base.,2022-05-01,2704266.0
2,Total. Total Nacional. Ninguna noche. Dato base.,2022-04-01,2499038.0
3,Total. Total Nacional. Ninguna noche. Dato base.,2022-03-01,2018328.0
4,Total. Total Nacional. Ninguna noche. Dato base.,2022-02-01,2146650.0


In [32]:
turism_stay_filtered['Origin'].value_counts()

Total. Total Nacional. Ninguna noche. Dato base.        81
Total. Total Nacional. 1 noche. Dato base.              81
Total. Total Nacional. De 2 a 3 noches. Dato base.      81
Total. Total Nacional. De 4 a 7 noches. Dato base.      81
Total. Total Nacional. De 8 a 15 noches. Dato base.     81
Total. Total Nacional. Más de 15 noches. Dato base.     81
Name: Origin, dtype: int64

In [33]:
turism_stay_filtered['Origin'] = turism_stay_filtered['Origin'].apply(lambda string: string.split(".")[2])
turism_stay_filtered['Origin'].value_counts()

 Ninguna noche       81
 1 noche             81
 De 2 a 3 noches     81
 De 4 a 7 noches     81
 De 8 a 15 noches    81
 Más de 15 noches    81
Name: Origin, dtype: int64

In [34]:
turism_stay_filtered.isna().sum()

Origin    0
Fecha     0
Valor     0
dtype: int64

In [35]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turists_stay.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_stay_filtered.to_excel(path+file_name)

### Estadística: Gasto de los turistas internacionales según comunidad autónoma de destino principal
#### Unidades: €, Millones €, Días, %, Tasas

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=10839

In [36]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/10839?date=20151001:20220601'

turism_expend_ccaa = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_expend_ccaa.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_expend_ccaa.to_excel(path+file_name)

In [37]:
turism_expend_ccaa.Nombre.unique()

array(['Gasto total. Total Nacional. Turista. Dato base. ',
       'Gasto total. Total Nacional. Turista. Tasa de variación anual. ',
       'Turista. Gasto total. Total Nacional. Acumulado en lo que va de año. ',
       'Turista. Gasto total. Total Nacional. Tasa de variación acumulada. ',
       'Gasto medio por persona. Total Nacional. Turista. Dato base. ',
       'Gasto medio por persona. Total Nacional. Turista. Tasa de variación anual. ',
       'Total Nacional. Turista. Gasto medio por persona. Acumulado en lo que va de año. ',
       'Total Nacional. Turista. Gasto medio por persona. Tasa de variación acumulada. ',
       'Gasto medio diario por persona. Total Nacional. Turista. Dato base. ',
       'Gasto medio diario por persona. Total Nacional. Turista. Tasa de variación anual. ',
       'Total Nacional. Turista. Gasto medio diario por persona. Acumulado en lo que va de año. ',
       'Total Nacional. Turista. Gasto medio diario por persona. Tasa de variación acumulada. ',


In [38]:
turism_expend_ccaa = turism_expend_ccaa.loc[turism_expend_ccaa['Nombre'].str.contains('base')].reset_index(drop=True)
turism_expend_ccaa.Nombre.unique()

array(['Gasto total. Total Nacional. Turista. Dato base. ',
       'Gasto medio por persona. Total Nacional. Turista. Dato base. ',
       'Gasto medio diario por persona. Total Nacional. Turista. Dato base. ',
       'Duración media de los viajes. Total Nacional. Turista. Dato base. ',
       'Andalucía. Turista. Gasto total. Dato base. ',
       'Andalucía. Turista. Gasto medio por persona. Dato base. ',
       'Andalucía. Turista. Gasto medio diario por persona. Dato base. ',
       'Andalucía. Turista. Duración media de los viajes. Dato base. ',
       'Balears, Illes. Turista. Gasto total. Dato base. ',
       'Balears, Illes. Turista. Gasto medio por persona. Dato base. ',
       'Balears, Illes. Turista. Gasto medio diario por persona. Dato base. ',
       'Balears, Illes. Turista. Duración media de los viajes. Dato base. ',
       'Canarias. Turista. Gasto total. Dato base. ',
       'Canarias. Turista. Gasto medio por persona. Dato base. ',
       'Canarias. Turista. Gasto med

In [39]:
turism_expend_ccaa_filtered = normalize_json(turism_expend_ccaa)
turism_expend_ccaa_filtered.head()

,Origin,Fecha,Valor
0,Gasto total. Total Nacional. Turista. Dato base.,2022-06-01,8992.53
1,Gasto total. Total Nacional. Turista. Dato base.,2022-05-01,8022.73
2,Gasto total. Total Nacional. Turista. Dato base.,2022-04-01,6900.61
3,Gasto total. Total Nacional. Turista. Dato base.,2022-03-01,5069.23
4,Gasto total. Total Nacional. Turista. Dato base.,2022-02-01,3756.86


In [40]:
turism_expend_ccaa_filtered['Origin'].value_counts()

Gasto total. Total Nacional. Turista. Dato base.                                     81
Gasto medio por persona. Total Nacional. Turista. Dato base.                         81
Otras Comunidades Autónomas. Turista. Gasto medio diario por persona. Dato base.     81
Otras Comunidades Autónomas. Turista. Gasto medio por persona. Dato base.            81
Otras Comunidades Autónomas. Turista. Gasto total. Dato base.                        81
Madrid, Comunidad de. Turista. Duración media de los viajes. Dato base.              81
Madrid, Comunidad de. Turista. Gasto medio diario por persona. Dato base.            81
Madrid, Comunidad de. Turista. Gasto medio por persona. Dato base.                   81
Madrid, Comunidad de. Turista. Gasto total. Dato base.                               81
Comunitat Valenciana. Turista. Duración media de los viajes. Dato base.              81
Comunitat Valenciana. Turista. Gasto medio diario por persona. Dato base.            81
Comunitat Valenciana. Turista. G

In [41]:
special_rows = turism_expend_ccaa_filtered.loc[turism_expend_ccaa_filtered['Origin'].str.contains('Nacional')]
special_rows_idx = special_rows.index
turism_expend_ccaa_filtered = turism_expend_ccaa_filtered.drop(special_rows_idx, axis = 0)

In [42]:
exp_type = []
origin = []
for string in special_rows['Origin']:
    string = string.split(".")
    exp_type.append(string[0])
    origin.append(string[1])


exp_type_ = []
origin_ = []
for i in turism_expend_ccaa_filtered['Origin']:
    i = i.split(".")
    exp_type_.append(i[2])
    origin_.append(i[0])
    
    
special_rows['Expenditure_type'] = exp_type
special_rows['Origin'] = origin
turism_expend_ccaa_filtered['Expenditure_type'] = exp_type_
turism_expend_ccaa_filtered['Origin'] = origin_

In [43]:
special_rows['Origin'].value_counts()
turism_expend_ccaa_filtered['Origin'].value_counts()

Andalucía                      324
Balears, Illes                 324
Canarias                       324
Cataluña                       324
Comunitat Valenciana           324
Madrid, Comunidad de           324
Otras Comunidades Autónomas    324
Name: Origin, dtype: int64

In [44]:
turism_expend_ccaa_clean = pd.concat([special_rows, turism_expend_ccaa_filtered], axis = 0)
display(turism_expend_ccaa_clean)

,Origin,Fecha,Valor,Expenditure_type
0,Total Nacional,2022-06-01,8992.53,Gasto total
1,Total Nacional,2022-05-01,8022.73,Gasto total
2,Total Nacional,2022-04-01,6900.61,Gasto total
3,Total Nacional,2022-03-01,5069.23,Gasto total
4,Total Nacional,2022-02-01,3756.86,Gasto total
...,...,...,...,...
2587,Otras Comunidades Autónomas,2016-02-01,6.86,Duración media de los viajes
2588,Otras Comunidades Autónomas,2016-01-01,10.46,Duración media de los viajes
2589,Otras Comunidades Autónomas,2015-12-01,10.01,Duración media de los viajes
2590,Otras Comunidades Autónomas,2015-11-01,7.10,Duración media de los viajes


In [45]:
turism_expend_ccaa_clean['Origin'].value_counts()

 Total Nacional                324
Andalucía                      324
Balears, Illes                 324
Canarias                       324
Cataluña                       324
Comunitat Valenciana           324
Madrid, Comunidad de           324
Otras Comunidades Autónomas    324
Name: Origin, dtype: int64

In [46]:
turism_expend_ccaa_clean.isna().sum()

Origin              0
Fecha               0
Valor               0
Expenditure_type    0
dtype: int64

In [47]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_expend_ccaa.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_expend_ccaa_clean.to_excel(path+file_name)

### Estadística: Número de etapas de los viajes y pernoctaciones según comunidad autónoma

Fuente: https://www.ine.es/jaxiT3/Datos.htm?t=23999

In [48]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/13949?date=20151001:20220601'

turism_nights_ccaa = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_nights_ccaa.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_nights_ccaa.to_excel(path+file_name)

In [49]:
turism_nights_ccaa.Nombre.unique()

array(['Total Nacional. Turista. Dato base. Etapas. ',
       'Total Nacional. Turista. Dato base. Pernoctaciones. ',
       'Total Nacional. Turista. Tasa de variación anual. Etapas. ',
       'Total Nacional. Turista. Tasa de variación anual. Pernoctaciones. ',
       'Andalucía. Turista. Dato base. Etapas. ',
       'Andalucía. Turista. Dato base. Pernoctaciones. ',
       'Andalucía. Turista. Tasa de variación anual. Etapas. ',
       'Andalucía. Turista. Tasa de variación anual. Pernoctaciones. ',
       'Balears, Illes. Turista. Dato base. Etapas. ',
       'Balears, Illes. Turista. Dato base. Pernoctaciones. ',
       'Balears, Illes. Turista. Tasa de variación anual. Etapas. ',
       'Balears, Illes. Turista. Tasa de variación anual. Pernoctaciones. ',
       'Canarias. Turista. Dato base. Etapas. ',
       'Canarias. Turista. Dato base. Pernoctaciones. ',
       'Canarias. Turista. Tasa de variación anual. Etapas. ',
       'Canarias. Turista. Tasa de variación anual. Pernoct

In [50]:
turism_nights_ccaa = turism_nights_ccaa.loc[turism_nights_ccaa['Nombre'].str.contains('Dato base. Pernoctaciones')]
turism_nights_ccaa.Nombre.unique()

array(['Total Nacional. Turista. Dato base. Pernoctaciones. ',
       'Andalucía. Turista. Dato base. Pernoctaciones. ',
       'Balears, Illes. Turista. Dato base. Pernoctaciones. ',
       'Canarias. Turista. Dato base. Pernoctaciones. ',
       'Cataluña. Turista. Dato base. Pernoctaciones. ',
       'Comunitat Valenciana. Turista. Dato base. Pernoctaciones. ',
       'Madrid, Comunidad de. Turista. Dato base. Pernoctaciones. ',
       'Otras Comunidades Autónomas. Turista. Dato base. Pernoctaciones. '],
      dtype=object)

In [51]:
turism_nights_ccaa_filtered = normalize_json(turism_nights_ccaa)
turism_nights_ccaa_filtered.head()

,Origin,Fecha,Valor
0,Total Nacional. Turista. Dato base. Pernoctaci...,2022-06-01,51536027.0
1,Total Nacional. Turista. Dato base. Pernoctaci...,2022-05-01,45405127.0
2,Total Nacional. Turista. Dato base. Pernoctaci...,2022-04-01,44527628.0
3,Total Nacional. Turista. Dato base. Pernoctaci...,2022-03-01,32273946.0
4,Total Nacional. Turista. Dato base. Pernoctaci...,2022-02-01,27616893.0


In [52]:
turism_nights_ccaa_filtered['Origin'] = turism_nights_ccaa_filtered['Origin'].apply(lambda string: string.split(".")[0])
turism_nights_ccaa_filtered['Origin'].unique()

array(['Total Nacional', 'Andalucía', 'Balears, Illes', 'Canarias',
       'Cataluña', 'Comunitat Valenciana', 'Madrid, Comunidad de',
       'Otras Comunidades Autónomas'], dtype=object)

In [53]:
turism_nights_ccaa_filtered.isna().sum()

Origin    0
Fecha     0
Valor     0
dtype: int64

In [54]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_nights_ccaa.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_nights_ccaa_filtered.to_excel(path+file_name)

### Estadística: Gasto de los turistas internacionales según país de residencia
Unidades: €, Millones €, Días, Tasas

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=23994

In [55]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/23994?date=20150101:20220601'

turism_expend_by_origin = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_expend_by_origin.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_expend_by_origin.to_excel(path+file_name)

In [56]:
turism_expend_by_origin.Nombre.unique()

array(['Total. Turista. Gasto total. Total Nacional. Dato base. ',
       'Total. Turista. Gasto total. Total Nacional. Tasa de variación anual. ',
       'Total. Turista. Gasto medio por persona. Total Nacional. Dato base. ',
       'Total. Turista. Gasto medio por persona. Total Nacional. Tasa de variación anual. ',
       'Total. Turista. Gasto medio diario por persona. Total Nacional. Dato base. ',
       'Total. Turista. Gasto medio diario por persona. Total Nacional. Tasa de variación anual. ',
       'Total. Turista. Duración media de los viajes. Total Nacional. Dato base. ',
       'Total. Turista. Duración media de los viajes. Total Nacional. Tasa de variación anual. ',
       'Alemania. Turista. Gasto total. Total Nacional. Dato base. ',
       'Alemania. Turista. Gasto total. Total Nacional. Tasa de variación anual. ',
       'Alemania. Turista. Gasto medio por persona. Total Nacional. Dato base. ',
       'Alemania. Turista. Gasto medio por persona. Total Nacional. Tasa de 

In [57]:
turism_expend_by_origin = turism_expend_by_origin.loc[turism_expend_by_origin['Nombre'].str.contains('Dato base.')]
turism_expend_by_origin.Nombre.unique()

array(['Total. Turista. Gasto total. Total Nacional. Dato base. ',
       'Total. Turista. Gasto medio por persona. Total Nacional. Dato base. ',
       'Total. Turista. Gasto medio diario por persona. Total Nacional. Dato base. ',
       'Total. Turista. Duración media de los viajes. Total Nacional. Dato base. ',
       'Alemania. Turista. Gasto total. Total Nacional. Dato base. ',
       'Alemania. Turista. Gasto medio por persona. Total Nacional. Dato base. ',
       'Alemania. Turista. Gasto medio diario por persona. Total Nacional. Dato base. ',
       'Alemania. Turista. Duración media de los viajes. Total Nacional. Dato base. ',
       'Bélgica. Turista. Gasto total. Total Nacional. Dato base. ',
       'Bélgica. Turista. Gasto medio por persona. Total Nacional. Dato base. ',
       'Bélgica. Turista. Gasto medio diario por persona. Total Nacional. Dato base. ',
       'Bélgica. Turista. Duración media de los viajes. Total Nacional. Dato base. ',
       'Francia. Turista. Gasto 

In [58]:
turism_expend_by_origin_filtered = normalize_json(turism_expend_by_origin)
turism_expend_by_origin_filtered.head()

,Origin,Fecha,Valor
0,Total. Turista. Gasto total. Total Nacional. D...,2021-01-01,34903.37
1,Total. Turista. Gasto total. Total Nacional. D...,2020-01-01,19786.78
2,Total. Turista. Gasto total. Total Nacional. D...,2019-01-01,91911.97
3,Total. Turista. Gasto total. Total Nacional. D...,2018-01-01,89750.75
4,Total. Turista. Gasto total. Total Nacional. D...,2017-01-01,87003.93


In [59]:
turism_expend_by_origin_filtered['Expenditure_type'] = turism_expend_by_origin_filtered['Origin']

turism_expend_by_origin_filtered['Origin'] = turism_expend_by_origin_filtered['Origin'].apply(lambda string: string.split(".")[0])
turism_expend_by_origin_filtered['Expenditure_type'] = turism_expend_by_origin_filtered['Expenditure_type'].apply(lambda string: string.split(".")[2])
turism_expend_by_origin_filtered.head()

,Origin,Fecha,Valor,Expenditure_type
0,Total,2021-01-01,34903.37,Gasto total
1,Total,2020-01-01,19786.78,Gasto total
2,Total,2019-01-01,91911.97,Gasto total
3,Total,2018-01-01,89750.75,Gasto total
4,Total,2017-01-01,87003.93,Gasto total


In [60]:
turism_expend_by_origin_filtered.isna().sum()

Origin              0
Fecha               0
Valor               0
Expenditure_type    0
dtype: int64

In [61]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_expend_by_origin.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_expend_by_origin_filtered.to_excel(path+file_name)

## Encuesta: Gasto de los turistas internacionales según tipo de alojamiento principal
Unidades: €, Millones €, Días, Tasas

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=10836

In [62]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/23993?date=20150101:20220601'

turism_expend_by_stay = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_expend_by_stay.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_expend_by_stay.to_excel(path+file_name)

In [63]:
turism_expend_by_stay.Nombre.unique()

array(['Turista. Total. Gasto total. Total Nacional. Dato base. ',
       'Turista. Total. Gasto total. Total Nacional. Tasa de variación anual. ',
       'Turista. Total. Gasto medio por persona. Total Nacional. Dato base. ',
       'Turista. Total. Gasto medio por persona. Total Nacional. Tasa de variación anual. ',
       'Turista. Total. Gasto medio diario por persona. Total Nacional. Dato base. ',
       'Turista. Total. Gasto medio diario por persona. Total Nacional. Tasa de variación anual. ',
       'Turista. Total. Duración media de los viajes. Total Nacional. Dato base. ',
       'Turista. Total. Duración media de los viajes. Total Nacional. Tasa de variación anual. ',
       'Turista. De mercado. Gasto total. Total Nacional. Dato base. ',
       'Turista. De mercado. Gasto total. Total Nacional. Tasa de variación anual. ',
       'Turista. De mercado. Gasto medio por persona. Total Nacional. Dato base. ',
       'Turista. De mercado. Gasto medio por persona. Total Nacional. 

In [64]:
turism_expend_by_stay = turism_expend_by_stay.loc[turism_expend_by_stay['Nombre'].str.contains('Dato base.')]

In [65]:
turism_expend_by_stay_filtered = normalize_json(turism_expend_by_stay)
turism_expend_by_stay_filtered.head()

,Origin,Fecha,Valor
0,Turista. Total. Gasto total. Total Nacional. D...,2021-01-01,34903.37
1,Turista. Total. Gasto total. Total Nacional. D...,2020-01-01,19786.78
2,Turista. Total. Gasto total. Total Nacional. D...,2019-01-01,91911.97
3,Turista. Total. Gasto total. Total Nacional. D...,2018-01-01,89750.75
4,Turista. Total. Gasto total. Total Nacional. D...,2017-01-01,87003.93


In [66]:
turism_expend_by_stay_filtered['Expenditure_type'] = turism_expend_by_stay_filtered['Origin'].apply(lambda string: string.split(".")[2])
turism_expend_by_stay_filtered['Origin'] = turism_expend_by_stay_filtered['Origin'].apply(lambda string: string.split(".")[1])
turism_expend_by_stay_filtered.head()

,Origin,Fecha,Valor,Expenditure_type
0,Total,2021-01-01,34903.37,Gasto total
1,Total,2020-01-01,19786.78,Gasto total
2,Total,2019-01-01,91911.97,Gasto total
3,Total,2018-01-01,89750.75,Gasto total
4,Total,2017-01-01,87003.93,Gasto total


In [67]:
turism_expend_by_stay_filtered.isna().sum()

Origin              0
Fecha               0
Valor               0
Expenditure_type    0
dtype: int64

In [68]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_expend_by_stay.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_expend_by_stay_filtered.to_excel(path+file_name)

## Encuesta: Gasto de los turistas internacionales según sexo y edad
Unidades: €, Millones €, Días, Tasas

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=24744

In [69]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/24744?date=20150101:20220601'
turism_expend_by_gender_sex = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_expend_by_gender_sex.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_expend_by_gender_sex.to_excel(path+file_name)

In [70]:
turism_expend_by_gender_sex.Nombre.unique()

array(['Turista. Gasto total. Total Nacional. Dato base. Total. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 15 a 24 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 25 a 44 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 45 a 64 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. 65 y más años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 15 a 24 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 25 a 44 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 45 a 64 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. 65 y más años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Mujeres. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Mujeres.

In [71]:
turism_expend_by_gender_sex = turism_expend_by_gender_sex.loc[turism_expend_by_gender_sex['Nombre'].str.contains('Dato base.')]

In [72]:
turism_expend_by_gender_sex.Nombre.unique()

array(['Turista. Gasto total. Total Nacional. Dato base. Total. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 15 a 24 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 25 a 44 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. De 45 a 64 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Total. 65 y más años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 15 a 24 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 25 a 44 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. De 45 a 64 años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Hombres. 65 y más años. ',
       'Turista. Gasto total. Total Nacional. Dato base. Mujeres. Todas las edades. ',
       'Turista. Gasto total. Total Nacional. Dato base. Mujeres.

In [73]:
turism_expend_by_gender_sex_filtered = normalize_json(turism_expend_by_gender_sex)

In [74]:
turism_expend_by_gender_sex_filtered['Sex'] = turism_expend_by_gender_sex_filtered['Origin'].apply(lambda string: string.split(".")[4])
turism_expend_by_gender_sex_filtered['Age'] = turism_expend_by_gender_sex_filtered['Origin'].apply(lambda string: string.split(".")[5])
turism_expend_by_gender_sex_filtered['Expenditure_type'] = turism_expend_by_gender_sex_filtered['Origin'].apply(lambda string: string.split(".")[1])
turism_expend_by_gender_sex_filtered['Origin'] = turism_expend_by_gender_sex_filtered['Origin'].apply(lambda string: string.split(".")[2])

turism_expend_by_gender_sex_filtered.head()

,Origin,Fecha,Valor,Sex,Age,Expenditure_type
0,Total Nacional,2021-01-01,34903.37,Total,Todas las edades,Gasto total
1,Total Nacional,2020-01-01,19786.78,Total,Todas las edades,Gasto total
2,Total Nacional,2019-01-01,91911.97,Total,Todas las edades,Gasto total
3,Total Nacional,2018-01-01,89750.75,Total,Todas las edades,Gasto total
4,Total Nacional,2017-01-01,87003.93,Total,Todas las edades,Gasto total


In [75]:
turism_expend_by_gender_sex_filtered = turism_expend_by_gender_sex_filtered.drop('Origin', axis=1)

In [76]:
turism_expend_by_gender_sex_filtered.isna().sum()

Fecha               0
Valor               0
Sex                 0
Age                 0
Expenditure_type    0
dtype: int64

In [77]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_expend_by_gender_sex.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_expend_by_gender_sex_filtered.to_excel(path+file_name)

## Encuesta: Gasto de los turistas internacionales según partidas de gasto
Unidades: €, Millones €, Días, Tasas

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=13938

In [78]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/23997?date=20150101:20220601'
turism_expend_by_theme = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_expend_by_theme.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_expend_by_theme.to_excel(path+file_name)

In [79]:
turism_expend_by_theme.Nombre.unique()

array(['Total Nacional. Turista. Gasto total. Dato base. ',
       'Total Nacional. Turista. Gasto total. Tasa de variación anual. ',
       'Turista. Gasto en paquete turístico. Total Nacional. Dato base. ',
       'Turista. Gasto en paquete turístico. Total Nacional. Tasa de variación anual. ',
       'Turista. Gasto no incluido en paquete turístico. Total Nacional. Dato base. ',
       'Turista. Gasto no incluido en paquete turístico. Total Nacional. Tasa de variación anual. ',
       'Turista. Gasto en transporte internacional. Total Nacional. Dato base. ',
       'Turista. Gasto en transporte internacional. Total Nacional. Tasa de variación anual. ',
       'Turista. Gasto en alojamiento. Total Nacional. Dato base. ',
       'Turista. Gasto en alojamiento. Total Nacional. Tasa de variación anual. ',
       'Turista. Gasto en manutención. Total Nacional. Dato base. ',
       'Turista. Gasto en manutención. Total Nacional. Tasa de variación anual. ',
       'Turista. Gasto en activi

In [80]:
turism_expend_by_theme = turism_expend_by_theme.loc[turism_expend_by_theme.Nombre.str.contains('Dato base.')]

In [81]:
turism_expend_by_theme.Nombre.unique()

array(['Total Nacional. Turista. Gasto total. Dato base. ',
       'Turista. Gasto en paquete turístico. Total Nacional. Dato base. ',
       'Turista. Gasto no incluido en paquete turístico. Total Nacional. Dato base. ',
       'Turista. Gasto en transporte internacional. Total Nacional. Dato base. ',
       'Turista. Gasto en alojamiento. Total Nacional. Dato base. ',
       'Turista. Gasto en manutención. Total Nacional. Dato base. ',
       'Turista. Gasto en actividades. Total Nacional. Dato base. ',
       'Turista. Otros gastos. Total Nacional. Dato base. '], dtype=object)

In [82]:
turism_expend_by_theme_filtered = normalize_json(turism_expend_by_theme)

In [83]:
turism_expend_by_theme_filtered.head()

,Origin,Fecha,Valor
0,Total Nacional. Turista. Gasto total. Dato base.,2021-01-01,34903.37
1,Total Nacional. Turista. Gasto total. Dato base.,2020-01-01,19786.78
2,Total Nacional. Turista. Gasto total. Dato base.,2019-01-01,91911.97
3,Total Nacional. Turista. Gasto total. Dato base.,2018-01-01,89750.75
4,Total Nacional. Turista. Gasto total. Dato base.,2017-01-01,87003.93


In [84]:
turism_expend_by_theme_filtered['Origin'] = turism_expend_by_theme_filtered['Origin'].apply(lambda string: string.split(".")[2])
turism_expend_by_theme_filtered = turism_expend_by_theme_filtered.rename(columns = {'Origin':'Expenditure_type'})
turism_expend_by_theme_filtered.head()

,Expenditure_type,Fecha,Valor
0,Gasto total,2021-01-01,34903.37
1,Gasto total,2020-01-01,19786.78
2,Gasto total,2019-01-01,91911.97
3,Gasto total,2018-01-01,89750.75
4,Gasto total,2017-01-01,87003.93


In [85]:
turism_expend_by_theme_filtered.isna().sum()

Expenditure_type    0
Fecha               0
Valor               0
dtype: int64

In [86]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_expend_by_theme.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_expend_by_theme_filtered.to_excel(path+file_name)

## Encuesta: Número de alojamientos turísticos y noches ocupados por residencia del viajero. Nacional y comunidades autónomas

Fuente: https://ine.es/jaxiT3/Tabla.htm?t=48423

In [87]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/48423?date=20181001:20211201'
turism_nights_apartments = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_nights_apartments.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_nights_apartments.to_excel(path+file_name)

In [88]:
turism_nights_apartments.Nombre.unique()

array(['Número de alojamientos turísticos ocupados. Total. Total Nacional. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Andalucía. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Aragón. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Asturias, Principado de. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Balears, Illes. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Canarias. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Cantabria. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Castilla y León. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Castilla - La Mancha. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Cataluña. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Comunitat Valenciana. Dato base. ',
       'Número de alojamiento

In [89]:
turism_nights_apartments_filtered = normalize_json(turism_nights_apartments)
turism_nights_apartments_filtered.head()

,Origin,Fecha,Valor
0,Número de alojamientos turísticos ocupados. To...,2021-12-01,460000.0
1,Número de alojamientos turísticos ocupados. To...,2021-11-01,403000.0
2,Número de alojamientos turísticos ocupados. To...,2021-10-01,583000.0
3,Número de alojamientos turísticos ocupados. To...,2021-09-01,596000.0
4,Número de alojamientos turísticos ocupados. To...,2021-08-01,890000.0


In [90]:
turism_nights_apartments_filtered['Category'] = turism_nights_apartments_filtered['Origin'].apply(lambda string: string.split(".")[0])
turism_nights_apartments_filtered['Residents'] = turism_nights_apartments_filtered['Origin'].apply(lambda string: string.split(".")[1])
turism_nights_apartments_filtered['CCAA'] = turism_nights_apartments_filtered['Origin'].apply(lambda string: string.split(".")[2])

turism_nights_apartments_filtered.head()

,Origin,Fecha,Valor,Category,Residents,CCAA
0,Número de alojamientos turísticos ocupados. To...,2021-12-01,460000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
1,Número de alojamientos turísticos ocupados. To...,2021-11-01,403000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
2,Número de alojamientos turísticos ocupados. To...,2021-10-01,583000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
3,Número de alojamientos turísticos ocupados. To...,2021-09-01,596000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
4,Número de alojamientos turísticos ocupados. To...,2021-08-01,890000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional


In [91]:
turism_nights_apartments_filtered.shape

(7020, 6)

In [92]:
turism_nights_apartments_filtered = turism_nights_apartments_filtered.drop('Origin', axis=1)
turism_nights_apartments_filtered.head()

,Fecha,Valor,Category,Residents,CCAA
0,2021-12-01,460000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
1,2021-11-01,403000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
2,2021-10-01,583000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
3,2021-09-01,596000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional
4,2021-08-01,890000.0,Número de alojamientos turísticos ocupados,Total,Total Nacional


In [93]:
turism_nights_apartments_filtered.isna().sum()

Fecha           0
Valor        1517
Category        0
Residents       0
CCAA            0
dtype: int64

In [94]:
missing_values = turism_nights_apartments_filtered.loc[turism_nights_apartments_filtered.Valor.isna()]

In [95]:
missing_values.count()

Fecha        1517
Valor           0
Category     1517
Residents    1517
CCAA         1517
dtype: int64

In [96]:
missing_values['CCAA'].value_counts()

 Melilla                        351
 Ceuta                          351
 Rioja, La                      143
 Navarra, Comunidad Foral de    119
 Extremadura                    110
 Asturias, Principado de         87
 Cantabria                       86
 Castilla - La Mancha            56
 Aragón                          47
 Murcia, Región de               45
 Castilla y León                 36
 Galicia                         30
 País Vasco                      29
 Balears, Illes                  17
 Canarias                         4
 Comunitat Valenciana             4
 Madrid, Comunidad de             2
Name: CCAA, dtype: int64

In [97]:
# No son hot topics, podemos eliminarlos.
missing_values_idx = missing_values.index
turism_nights_apartments_filtered = turism_nights_apartments_filtered.drop(missing_values_idx, axis=0).reset_index(drop=True)

In [98]:
turism_nights_apartments_filtered.isna().sum()

Fecha        0
Valor        0
Category     0
Residents    0
CCAA         0
dtype: int64

In [99]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_nights_apartments.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_nights_apartments_filtered.to_excel(path+file_name)

## Encuesta: Número de alojamientos turísticos y noches ocupados por residencia del viajero. Puntos turísticos.

Fuente: https://ine.es/jaxiT3/Tabla.htm?t=48424

In [100]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/48424?date=20181001:20211201'
turism_nights_apartments_hot_spots = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_nights_apartments_hot_spots.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_nights_apartments_hot_spots.to_excel(path+file_name)

In [101]:
turism_nights_apartments_hot_spots.Nombre.unique()

array(['Número de alojamientos turísticos ocupados. Total. Alicante/Alacant. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Barcelona. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Benalmádena. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Benidorm. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Bilbao. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Cádiz. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Cartagena. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Chiclana de la Frontera. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Córdoba. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Coruña, A. Dato base. ',
       'Número de alojamientos turísticos ocupados. Total. Donostia/San Sebastián. Dato base. ',
       'Número de alojamientos turísticos ocupados

In [102]:
turism_nights_apartments_hot_spots_filtered = normalize_json(turism_nights_apartments_hot_spots)

In [103]:
print(turism_nights_apartments_hot_spots_filtered.shape)
turism_nights_apartments_hot_spots_filtered.head()

(13689, 3)


,Origin,Fecha,Valor
0,Número de alojamientos turísticos ocupados. To...,2021-12-01,6000.0
1,Número de alojamientos turísticos ocupados. To...,2021-11-01,5000.0
2,Número de alojamientos turísticos ocupados. To...,2021-10-01,8000.0
3,Número de alojamientos turísticos ocupados. To...,2021-09-01,8000.0
4,Número de alojamientos turísticos ocupados. To...,2021-08-01,12000.0


In [104]:
turism_nights_apartments_hot_spots_filtered['Category'] = turism_nights_apartments_hot_spots_filtered['Origin'].apply(lambda x: x.split('.')[0])
turism_nights_apartments_hot_spots_filtered['Residents'] = turism_nights_apartments_hot_spots_filtered['Origin'].apply(lambda x: x.split('.')[1])
turism_nights_apartments_hot_spots_filtered['Site'] = turism_nights_apartments_hot_spots_filtered['Origin'].apply(lambda x: x.split('.')[2])

turism_nights_apartments_hot_spots_filtered = turism_nights_apartments_hot_spots_filtered.drop('Origin', axis=1)
turism_nights_apartments_hot_spots_filtered.head()

,Fecha,Valor,Category,Residents,Site
0,2021-12-01,6000.0,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
1,2021-11-01,5000.0,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
2,2021-10-01,8000.0,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
3,2021-09-01,8000.0,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
4,2021-08-01,12000.0,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant


In [105]:
turism_nights_apartments_hot_spots_filtered.isna().sum()

Fecha           0
Valor        4735
Category        0
Residents       0
Site            0
dtype: int64

In [106]:
missing_values = turism_nights_apartments_hot_spots_filtered.loc[turism_nights_apartments_hot_spots_filtered['Valor'].isna()]
missing_values

,Fecha,Valor,Category,Residents,Site
19,2020-05-01,NaN,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
20,2020-04-01,NaN,Número de alojamientos turísticos ocupados,Total,Alicante/Alacant
86,2021-04-01,NaN,Número de alojamientos turísticos ocupados,Total,Benalmádena
87,2021-03-01,NaN,Número de alojamientos turísticos ocupados,Total,Benalmádena
88,2021-02-01,NaN,Número de alojamientos turísticos ocupados,Total,Benalmádena
...,...,...,...,...,...
13683,2019-03-01,NaN,Estancia media,Residentes en el Extranjero,Zaragoza
13684,2019-02-01,NaN,Estancia media,Residentes en el Extranjero,Zaragoza
13685,2019-01-01,NaN,Estancia media,Residentes en el Extranjero,Zaragoza
13686,2018-12-01,NaN,Estancia media,Residentes en el Extranjero,Zaragoza


In [107]:
missing_values.Site.value_counts()
missing_values.Category.value_counts()

Número de alojamientos turísticos ocupados    2025
Estancia media                                2025
Número de noches ocupadas                      685
Name: Category, dtype: int64

In [108]:
missing_values_idx = missing_values.index

turism_nights_apartments_hot_spots_filtered = turism_nights_apartments_hot_spots_filtered.drop(missing_values_idx, axis=0)

In [109]:
turism_nights_apartments_hot_spots_filtered.isna().sum()

Fecha        0
Valor        0
Category     0
Residents    0
Site         0
dtype: int64

In [110]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_nights_apartments_hot_spots.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_nights_apartments_hot_spots_filtered.to_excel(path+file_name)

## Encuesta: Encuesta de Ocupación Hotelera. Viajeros y pernoctaciones por puntos turísticos


Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=2078

In [111]:
url = 'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/49370?date=20151001:20220601'

turism_nights_hotels_hot_spot = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_nights_hotels_hot_spot.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_nights_hotels_hot_spot.to_excel(path+file_name)

In [112]:
turism_nights_hotels_hot_spot.head()

,COD,Nombre,FK_Unidad,FK_Escala,Data
0,EOT2611,Nacional. Viajeros. Vitoria-Gastéiz. Residente...,213,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
1,EOT2612,Nacional. Viajeros. Vitoria-Gastéiz. Residente...,213,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
2,EOT2613,Nacional. Pernoctaciones. Vitoria-Gastéiz. Res...,242,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
3,EOT2614,Nacional. Pernoctaciones. Vitoria-Gastéiz. Res...,242,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."
4,EOT2615,Nacional. Viajeros. Albacete. Residentes en Es...,213,1,"[{'Fecha': 1654034400000, 'FK_TipoDato': 2, 'F..."


In [113]:
turism_nights_hotels_hot_spot = turism_nights_hotels_hot_spot.loc[turism_nights_hotels_hot_spot.Nombre.str.contains('hoteleros')]
turism_nights_hotels_hot_spot.Nombre.unique()

array(['Nacional. Establecimientos hoteleros. Viajeros. 03063-Denia. Residentes en España.',
       'Nacional. Establecimientos hoteleros. Viajeros. 03063-Denia. Residentes en el extranjero.',
       'Nacional. Establecimientos hoteleros. Pernoctaciones. 03063-Denia. Residentes en España.',
       'Nacional. Establecimientos hoteleros. Pernoctaciones. 03063-Denia. Residentes en el extranjero.',
       'Nacional. Establecimientos hoteleros. Viajeros. 04066-Níjar. Residentes en España.',
       'Nacional. Establecimientos hoteleros. Viajeros. 04066-Níjar. Residentes en el extranjero.',
       'Nacional. Establecimientos hoteleros. Pernoctaciones. 04066-Níjar. Residentes en España.',
       'Nacional. Establecimientos hoteleros. Pernoctaciones. 04066-Níjar. Residentes en el extranjero.',
       'Nacional. Establecimientos hoteleros. Viajeros. 07014-Capdepera. Residentes en España.',
       'Nacional. Establecimientos hoteleros. Viajeros. 07014-Capdepera. Residentes en el extranjero.',
   

In [114]:
turism_nights_hotels_hot_spot_filtered = normalize_json(turism_nights_hotels_hot_spot)
turism_nights_hotels_hot_spot_filtered.head()

,Origin,Fecha,Valor,Notas
0,Nacional. Establecimientos hoteleros. Viajeros...,2022-06-01,13580.0,NaN
1,Nacional. Establecimientos hoteleros. Viajeros...,2022-05-01,10317.0,NaN
2,Nacional. Establecimientos hoteleros. Viajeros...,2022-04-01,8339.0,NaN
3,Nacional. Establecimientos hoteleros. Viajeros...,2022-03-01,NaN,NaN
4,Nacional. Establecimientos hoteleros. Viajeros...,2022-02-01,NaN,NaN


In [115]:
turism_nights_hotels_hot_spot_filtered['Category'] = turism_nights_hotels_hot_spot_filtered['Origin'].apply(lambda x: x.split(".")[4])
turism_nights_hotels_hot_spot_filtered['Site'] = turism_nights_hotels_hot_spot_filtered['Origin'].apply(lambda x: x.split(".")[3])
turism_nights_hotels_hot_spot_filtered['Origin'] = turism_nights_hotels_hot_spot_filtered['Origin'].apply(lambda x: x.split(".")[1])

turism_nights_hotels_hot_spot_filtered = turism_nights_hotels_hot_spot_filtered.rename(columns = {'Origin':'Establishment'})

In [117]:
turism_nights_hotels_hot_spot_filtered.isna().sum()

Establishment       0
Fecha               0
Valor            1962
Notas            6362
Category            0
Site                0
dtype: int64

In [119]:
missing_values = turism_nights_hotels_hot_spot_filtered.loc[turism_nights_hotels_hot_spot_filtered['Valor'].isna()].index
turism_nights_hotels_hot_spot_filtered = turism_nights_hotels_hot_spot_filtered.drop(missing_values, axis=0)

In [120]:
turism_nights_hotels_hot_spot_filtered = turism_nights_hotels_hot_spot_filtered.drop('Notas', axis=1)

In [121]:
turism_nights_hotels_hot_spot_filtered.isna().sum()

Establishment    0
Fecha            0
Valor            0
Category         0
Site             0
dtype: int64

In [122]:
# Salvamos el dataset limpio como archivo Excel
file_name = 'turism_nights_hotels_hot_spot_filtered.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/clean/'

turism_nights_hotels_hot_spot_filtered.to_excel(path+file_name)

## Encuesta: Encuesta de Ocupación en Campings. Viajeros y pernoctaciones por puntos turísticos

Fuente: https://www.ine.es/jaxiT3/Tabla.htm?t=2047

In [133]:
url = 'https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/2047?tip=AM&'
turism_nights_camping_hot_spot = pd.read_json(url)

# Salvamos el dataset sucio como Excel
file_name = 'turism_nights_camping_hot_spot.xlsx'
path = 'C:/Users/ameli/IRONHACK/WEEK_5/MBP/data/raw/'

turism_nights_camping_hot_spot.to_excel(path+file_name)

In [134]:
turism_nights_camping_hot_spot.head()

,COD,Nombre,T3_Unidad,T3_Escala,MetaData,Data
0,EOT7511,Nacional. Viajeros. Andalucía: Costa De Almerí...,Viajeros,,"[{'Id': 284332, 'Variable': {'Id': 433, 'Nombr...","[{'Fecha': '2022-06-01T00:00:00.000+02:00', 'T..."
1,EOT7512,Nacional. Viajeros. Andalucía: Costa De Almerí...,Viajeros,,"[{'Id': 284332, 'Variable': {'Id': 433, 'Nombr...","[{'Fecha': '2022-06-01T00:00:00.000+02:00', 'T..."
2,EOT7513,Nacional. Pernoctaciones. Andalucía: Costa De ...,Pernoctaciones,,"[{'Id': 284333, 'Variable': {'Id': 433, 'Nombr...","[{'Fecha': '2022-06-01T00:00:00.000+02:00', 'T..."
3,EOT7514,Nacional. Pernoctaciones. Andalucía: Costa De ...,Pernoctaciones,,"[{'Id': 284333, 'Variable': {'Id': 433, 'Nombr...","[{'Fecha': '2022-06-01T00:00:00.000+02:00', 'T..."
4,EOT7523,Nacional. Viajeros. Andalucía: Costa De La Luz...,Viajeros,,"[{'Id': 284332, 'Variable': {'Id': 433, 'Nombr...","[{'Fecha': '2022-06-01T00:00:00.000+02:00', 'T..."


In [135]:
turism_nights_camping_hot_spot = turism_nights_camping_hot_spot.loc[turism_nights_camping_hot_spot.Nombre.str.contains('Pernoctaciones')]
turism_nights_camping_hot_spot.Nombre.unique()

array(['Nacional. Pernoctaciones. Andalucía: Costa De Almería. Residentes en España.',
       'Nacional. Pernoctaciones. Andalucía: Costa De Almería. Residentes en el extranjero.',
       'Nacional. Pernoctaciones. Andalucía: Costa De La Luz(Huelva). Residentes en España.',
       'Nacional. Pernoctaciones. Andalucía: Costa De La Luz(Huelva). Residentes en el extranjero.',
       'Nacional. Pernoctaciones. Andalucía: Costa De La Luz De Cádiz. Residentes en España.',
       'Nacional. Pernoctaciones. Andalucía: Costa De La Luz De Cádiz. Residentes en el extranjero.',
       'Nacional. Pernoctaciones. Andalucía: Costa Del Sol (Málaga). Residentes en España.',
       'Nacional. Pernoctaciones. Andalucía: Costa Del Sol (Málaga). Residentes en el extranjero.',
       'Nacional. Pernoctaciones. Andalucía: Costa Tropical (Granada). Residentes en España.',
       'Nacional. Pernoctaciones. Andalucía: Costa Tropical (Granada). Residentes en el extranjero.',
       'Nacional. Pernoctaciones. Ast

KeyError: "['FK_TipoDato', 'FK_Periodo', 'Secreto'] not found in axis"